In [140]:
def topleft(box):
    return np.array([box[1],box[2]],np.float32)
def botright(box):
    return np.array([box[3],box[4]],np.float32)
def corners(box):
    z = np.zeros(4,np.float32)
    z[:2] = topleft(box)
    z[2:4] = botright(box)
    return z
def get_distance(v1,v2):
    
    if(len(v1)==len(v2)):
        dist = distance.euclidean(np.array(v1),np.array(v2))
    else:
        dist = -1
    if(dist>1000):
        dist=30
    return dist
def bounding_box_naive(points):
    """returns a list containing the bottom left and the top right 
    points in the sequence
    Here, we use min and max four times over the collection of points
    """
    

    top_left_x = min(point[0][0] for point in points)
    top_left_y = min(point[0][1] for point in points)
    bot_right_x = max(point[0][0] for point in points)
    bot_right_y = max(point[0][1] for point in points)

    center_x = (top_left_x+bot_right_x)/2
    center_y = (top_left_y+bot_right_y)/2
    return [center_x,center_y,bot_right_x-top_left_x,bot_right_y-top_left_y]

In [ ]:
class Detection(object):
    def __init__(self,conf,bbox,desc=[]):
        self.xmin = bbox[0]
        self.ymin = bbox[1]
        self.xmax = bbox[2]
        self.ymax = bbox[3]
        self.conf = conf
        self.description = desc
        

In [154]:
class Track(object):
    def __init__(self,method,_id,bbox,desc=[]):
        self.conf = 0.3
        self.xmin = bbox[1]
        self.ymin = bbox[2]
        self.xmax = bbox[3]
        self.ymax = bbox[4]
        self.pred_xmin = bbox[1]
        self.pred_ymin = bbox[2]
        self.pred_xmax = bbox[3]
        self.pred_ymax = bbox[4]
        self.tracked_count = 1
        self.missed_count=0
        self.matched=True
        self.descriptor = desc
        self.track_id = _id
        self.method = method
        if(method=='kalman_corners'):
            self.init_kalman_tracker()
    def init_kalman_tracker(self):
        self.kalman_tracker = cv.KalmanFilter(8,4)
        kalman_tracker.measurementMatrix = np.array([[1,0,0,0,0,0,0,0],[0,1,0,0,0,0,0,0],[0,0,1,0,0,0,0,0],[0,0,0,1,0,0,0,0]],np.float32)

        kalman_tracker.transitionMatrix = np.array([[1,0,0,0,1,0,0,0],[0,1,0,0,0,1,0,0],[0,0,1,0,0,0,1,0],[0,0,0,1,0,0,0,1]
                                           ,[0,0,0,0,1,0,0,0],[0,0,0,0,0,1,0,0],[0,0,0,0,0,0,1,0],[0,0,0,0,0,0,0,1]],np.float32)

        kalman_tracker.processNoiseCov = np.array([[1,0,0,0,0,0,0,0],[0,1,0,0,0,0,0,0],[0,0,1,0,0,0,0,0],[0,0,0,1,0,0,0,0]
                                          ,[0,0,0,0,1,0,0,0],[0,0,0,0,0,1,0,0],[0,0,0,0,0,0,1,0],[0,0,0,0,0,0,0,1]],np.float32)*0.001

        kalman_tracker.predict();

        kalman_tracker.correct(initial)
        kalman_tracker.predict();
        kalman_tracker.correct(initial)
        kalman_tracker.predict();
        kalman_tracker.correct(initial)
        kalman_tracker.predict();
        kalman_tracker.correct(initial)
        
    
    def corners(self):
        
        return np.array([self.xmin,self.ymin,self.xmax,self.ymax],np.float32)
    def copy(self):
        other = Track(self.track_id,np.array([0,self.conf,self.xmin,self.ymin,self.xmax,self.ymax],np.float32),self.descriptor)
        other.tracked_count = self.tracked_count
        other.missed_count = self.missed_count
        other.matched = self.matched
        
        return other
    def center(self):
    
        return np.array([(self.xmin+self.xmax)/2,(self.ymin+self.ymax)/2],np.float32)
    def update(self,det,desc,frame_gray,prev_frame_gray):
        self.xmin = det[1]
        self.ymin = det[2]
        self.xmax=det[3]
        self.ymax = det[4]
        if(len(desc)!=self.descriptor.shape[0]):
            self.descriptor = []
            self.descriptor = np.zeros(len(desc),np.float32)
        self.descriptor[:] = desc
        self.matched = True
        self.missed_count = 0
        self.tracked_count +=1
        self.predict(prev_frame_gray,frame_gray)
        if(self.tracked_count>3):
            self.conf = det[0]
        if(self.method=='kalman_corners'):
            self.kalman_tracker.correct(self.corners())
    def apply_prediction(self):
        self.xmin = self.pred_xmin
        self.ymin = self.pred_ymin
        self.xmax = self.pred_xmax
        self.ymax = self.pred_ymax
    def shiftKeyPointsFlow(self,frame,prev_frame):
        frame_grey = cv.cvtColor(frame,cv.COLOR_BGR2GRAY)
        prev_frame_grey = cv.cvtColor(prev_frame,cv.COLOR_BGR2GRAY)
        mask = np.zeros(frame_grey.shape, dtype = "uint8")
        cv.rectangle(mask, (int(self.xmin), int(self.ymin)), (int(self.xmax), int(self.ymax)), (255, 255, 255), -1)
        p0 = cv.goodFeaturesToTrack(prev_frame_grey, mask = mask, **feature_params)

        if(not p0 is None ):
            p1, st, err = cv.calcOpticalFlowPyrLK(prev_frame_grey, frame_grey,p0, None, **lk_params)
            old_box = bounding_box_naive(p0)
            new_box = bounding_box_naive(p1)
            offset = [new_box[0]-old_box[0],new_box[1]-old_box[1]]
            new_center = self.center() +offset
            old_width = self.xmax - self.xmin
            old_height = self.ymax-self.ymin

            new_width = old_width * (new_box[2]/old_box[2])

            new_height = old_height * (new_box[3]/old_box[3])

            if(new_width==0 or new_width>frame_width or math.isnan(new_width)):
                new_width=0
            if(new_height==0 or new_height>frame_height or math.isnan(new_height)):
                new_height=0

            self.pred_xmin = new_center[0] - (new_width/2)
            self.pred_ymin=new_center[1] - (new_height/2)
            self.pred_xmax=new_center[0] + (new_width/2)
            self.pred_ymax=new_center[1]+ (new_height/2)
            
        elif(self.missed_count>0):
            self.conf=0
    def predict(self,prev_frame_gray,frame_gray):
        if(self.method=='kalman_corners'):
            pred = self.kalman_tracker.predict()
            self.pred_xmin = pred[0][0]
            self.pred_ymin = pred[1][0]
            self.pred_xmax = pred[2][0]
            self.pred_ymax = pred[3][0]
            
        elif(self.method =='keypoint_flow'):
            self.shiftKeyPointsFlow(frame_gray,prev_frame_gray)
           
            
    

In [155]:
class Tracker(object):
    def __init__(self,method='keypoint_flow'):
        self.tracking_method = method
        self.tracks = []
        self.cur_id=1
        self.frameCount =1
    def get_distance_matrix(self,dets):
        dists = np.zeros((len(dets),len(self.tracks)),np.float32)
        for itrack in range(len(self.tracks)):
            for ipred in range(len(dets)):
                iou_dist = (1- iou(corners(dets[ipred]),self.tracks[itrack].corners(),coords='corners'))
                desc_dist = get_distance(self.tracks[itrack].descriptor,dets[ipred][6:])
                iou_overlap = iou(corners(dets[ipred]),self.tracks[itrack].corners(),coords='corners')
                if(iou_dist==1):
                    iou_dist=3
                if(iou_dist<0.7):
                    iou_dist = 0
                dists[ipred,itrack] = iou_dist
        return dists
    def track(self,dets,frame_gray,prev_frame_gray):
        
        dists = self.get_distance_matrix(dets)
        matched_indices = linear_assignment(dists)
        for m in matched_indices:
            descr_t = self.tracks[m[1]].descriptor
            descr_p = preds[m[0]][6:]

            if(self.tracks[m[1]].missed_count<3):
                iou_threshold=2
            elif(self.tracks[m[1]].missed_count<8):
                iou_threshold=2.5
            else:
                iou_threshold=3
            if(dists[m[0],m[1]]>2):#iou_threshold):
                m[0] = -1
                m[1]=-1
        for trk in self.tracks:
            trk.matched=False
            
        for d,det in enumerate(dets):
            if(d not in matched_indices[:,0] ):
                
                temp_pred = np.copy(det)
               
                self.tracks.append(Track(self.tracking_method,self.cur_id,temp_pred,temp_pred[6:]))
                
                self.cur_id+=1
                
            else:
                index = np.where(matched_indices[:,0]==d)
                index = matched_indices[index][0][1]
                self.tracks[index].update(det,dets[6:],frame_gray,prev_frame_gray)
        
        for t,trk in enumerate(self.tracks):
            if(t not in matched_indices[:,1] and trk.matched==False):
                trk.missed_count+=1
                if(trk.tracked_count<10):
                    if(trk.missed_count>4):
                        trk.conf = 0.3 #hide
                    elif(trk.missed_count>6):
                        trk.conf = 0.2 #remove
                elif(trk.tracked_count<30):
                    if(trk.missed_count>8):
                        trk.conf=0.3 #hide
                    elif(trk.missed_count>12):
                        trk.conf = 0.2 #remove
                else:
                    if(trk.missed_count>15):
                        trk.conf=0.3 #hide
                    elif(trk.missed_count>20):
                        trk.conf = 0.2 #remove
                if(trk.missed_count>30):
                    trk.conf=0 #remove
                
                trk.apply_prediction()
       
        self.frameCount+=1
    def get_display_tracks(self):
        return [track for track in self.tracks if track.conf>0.2]

In [156]:
def load_detections(dataset,detector):
    text_file_path = "detections/%s/%s.txt"%(dataset,detector)
    f = open(text_file_path,"r")
    line = f.readline()
    detections={}
    comps = []
    while(line):

        line = line.replace("\n", "")
        comps = line.split(",")
        
        if(comps[0] in detections):
            detections[comps[0]].append(list(map(float, comps[2:])))
        else:
            detections[comps[0]]=[]
            detections[comps[0]].append(list(map(float, comps[2:])))
        line=f.readline()
    f.close()
    return detections


In [157]:
from bounding_box_utils.bounding_box_utils import iou
import numpy as np
import json
import math
import scipy.interpolate as interp
from scipy.spatial import distance
from imutils.video import VideoStream
import argparse
import datetime
import motmetrics as mm
import imutils
import time
from sklearn import preprocessing
import cv2 as cv
import time
from sort import *
from sklearn.utils.linear_assignment_ import linear_assignment
import json
from bounding_box_utils.bounding_box_utils import iou

acc = mm.MOTAccumulator(auto_id=True)
feature_params = dict( maxCorners = 30,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))
iou_overlaps = []
desc_dists = []
confusion_frames = []
confusion_tracks = []
confusion_distances =[]
colors = [[255,0,0],[0,255,0],[0,0,255],[0,128,128],[128,0,128],[128,128,0],[255,0,0],[0,255,0],[0,0,255],[0,128,128],[128,0,128],[128,128,0],
         [255,0,0],[0,255,0],[0,0,255],[0,128,128],[128,0,128],[128,128,0],[255,0,0],[0,255,0],[0,0,255],[0,128,128],[128,0,128],[128,128,0],[255,0,0],[0,255,0],[0,0,255],[0,128,128],[128,0,128],[128,128,0]]
tracking_methods=['keypoint_flow']
#tracking_methods=['center_flow','keypoint_flow','kalman_center','kalman_corners','SORT']
detectors = ['yolo']
#detectors = ['ssd300','retinanet','yolo']
#'center_flow','keypoint_flow','kalman_center','kalman_corners',
datasets=['graal_1']
times = {}
for dataset in datasets:
    times[dataset]={}
    images_input_path='../%s/'%dataset
    image_id_prefix= dataset
    frame_width=1032
    frame_height=778
    if(dataset=='venc'):
        frame_width = 1280
        frame_height = 960
    if(dataset=='modd'):
        frame_width=640
        frame_height=464
    if(dataset=='mot_1'):
        frame_width=768
        frame_height=576
    iou_threshold = 0.1
    for detector in detectors:
        times[dataset][detector] = {}
        boat_class=8
        min_conf=0.6
        if(detector=='ssd300'):
            boat_class=4
            min_conf=0.5
        if(detector=='def'):
            boat_class=1
        

        path = '%s/%s_videos'%(detector,image_id_prefix)
        detections = load_detections(image_id_prefix,detector)
        for tracking_method in tracking_methods:
            times[dataset][detector][tracking_method] = []
            video_output_path='%s/%s.avi'%(path,tracking_method)
            json_output_path='%s/%s.json'%(path,tracking_method)
            out_tracking = cv.VideoWriter(video_output_path,cv.VideoWriter_fourcc('M','J','P','G'), 30, (frame_width,frame_height))
            frameCount =0
            no_tracking_res = [] 
            tracking_res = []
            kalman_trackers=[]
            # initialize the first frame in the video stream
            frameCount =0
            step_up = 0.1
            step_down = 0.2
            print('Running: Dataset:%s, Detector:%s, Tracker:%s, @%dx%d'%(dataset,detector,tracking_method,frame_width,frame_height))
            preds = []
            tracks=[]
            started = False
            multiplier=0
            cc=0
            prev_frame=None
            
            total_frames=641
            if(tracking_method=='SORT'):
                mot_tracker = Sort()
            else:
                tracker_wrapper = Tracker(tracking_method)
            
            while frameCount<total_frames:
                
                # grab the current frame and initialize the occupied/unoccupied
                # text
                frame = cv.imread('%s%s.jpg'%(images_input_path,str(frameCount+1).zfill(5)))
                
                
                # if the frame could not be grabbed, then we have reached the end
                # of the video
                if frame is None:
                    break

                if(frameCount<0):
                    continue

                preds = []
                if '%s/%s.jpg'%(image_id_prefix,str(frameCount+1).zfill(5)) in detections:

                    for box in detections['%s/%s.jpg'%(image_id_prefix,str(frameCount+1).zfill(5))]:

                        if(box[0]!=boat_class or box[1]<min_conf):

                            continue
                        if(tracking_method=='SORT'):
                            temp_pred = box[2:]

                            temp_pred = np.insert(temp_pred,4,box[1])

                            preds.append(temp_pred)
                        else :
                            
                            
                            
                            preds.append(box[1:])
                start= time.time()
                if(tracking_method=='SORT'):

                    preds = np.asarray(preds)
                    trackers = mot_tracker.update(preds)
                    to_display = []
                    for itrk,tracker in enumerate(trackers):
                        to_display.append([tracker[4],boat_class,preds[itrk][4],tracker[0],tracker[1],tracker[2],tracker[3]])
                else:   
                    
                    tracker_wrapper.track(preds,frame,prev_frame)
                    
                    
                    to_display = tracker_wrapper.get_display_tracks()
                    




                        #print(acc.mot_events.loc[frameId])
               
                i=0
                for box in to_display:
                    
                # Transform the predicted bounding boxes for the 512x512 image to the original image dimensions.
                    if(box.conf<0.5):
                        i+=1
                        continue
                    xmin = int(box.xmin)
                    ymin = int(box.ymin)
                    xmax =int(box.xmax)
                    ymax =int(box.ymax)
                    
                    cv.rectangle(frame, (int(xmin), int(ymin)), (int(xmax),int(ymax)), colors[int(box.track_id)%len(colors)], 2)

                    if(tracking_method=='kalman_center'):
                        cv.circle(frame,(int(predictions[i][0][0]),int(predictions[i][1][0])),5,(255,0,0),2)
                    #if(tracking_method=='kalman_corners'):
                        #cv.circle(frame,(int(predictions[i][0][0]),int(predictions[i][1][0])),5,(255,0,0),2)
                        #cv.circle(frame,(int(predictions[i][2][0]),int(predictions[i][3][0])),5,(255,0,0),2)
                    cv.putText(frame,'{:.2f}'.format( box.conf), (xmin, ymin),cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)


                    tracking_res.append({"image_id" : frameCount+1, "category_id" : 1, "bbox" : [float(xmin),float(ymin),float(xmax-xmin),float(ymax-ymin)], "score" : np.minimum(1.0,box.conf),"id":box.track_id})
                    #f.write("graal_2/%s.jpg,%s,%d,%f,%f,%f,%f,%f\n"%(str(frameCount+1).zfill(5),classes[int(box[1])],box[1],box[2],xmin,ymin,xmax,ymax))
                    i+=1
                    times[dataset][detector][tracking_method].append(time.time()-start)
                
                out_tracking.write(frame)
                #cv.imwrite('debug_frames/%s.jpg'%str(frameCount+1),frame)
                frameCount+=1
                prev_frame=frame
                flow = None
            # cleanup the camera and close any open windows

            out_tracking.release()


            with open(json_output_path, 'w') as outfile:  
                json.dump(tracking_res, outfile)

Running: Dataset:graal_1, Detector:yolo, Tracker:keypoint_flow, @1032x778


C:\Users\issam\Anaconda3\lib\site-packages\ipykernel_launcher.py:92: RuntimeWarning: divide by zero encountered in float_scalars
